# Analysis of the ''EarSet'' dataset from the paper ''EarSet: A Multi-Modal Dataset for Studying the Impact of Head and Facial Movements  on In-Ear PPG Signals''

Paper:  https://www.nature.com/articles/s41597-023-02762-3<br />     
Note on data preprocessing from the paper: "The data recorded from the Zephyr does not require additional processing as they
are already pre-processed (with the exception of the ECGAmplitude and the BRAmplitude, which can be easily
pre-processed using NeuroKit library). However, the data collected from our earable prototype requires pre-processing. Firstly, the raw accelerometer
data has to be converted to milli-g units by multiplying with 0.061, and the raw gyroscope data has to be converted
to milli-dps (degrees per second) by multiplying with 17.5. This converts the raw IMU sensor data from
an integer format to a more usable/standard format (i.e., milli-g and milli-dps). We then remove the direct current
(DC) offset from the gyroscope data by applying a Butterworth band-pass filter (0.4–4 Hz cutoff). Secondly,
the PPG signals can be pre-processed using bandpass filtering options available in Heartpy or NeuroKit libraries
to extract HR, SpO2, etc."

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt

In [ ]:
# Get the list of all files and directories in P0 directory
path = '../data/Dataset/P0/EARBUDS/'
dir_list = os.listdir(path)

# Store the action name for the files in a list
action_name_for_file = ['still', 'nod', 'shake', 'tilt', 'eyes-ud', 'eyes-lr', 'brow-raiser', 'brow-lowerer', 'wink-r',
                        'wink-l', 'lip-puller', 'chin-raiser', 'mouth-stretch-', 'chewing', 'speaking', 'walking', 'running']

# Define an action mapping table with the actions from the paper
actions = ['Still', 'Nod', 'Shake', 'Tilt', 'Vertical Eyes Movements', 'Horizontal Eyes Movements', 'Brow Raiser', 'Brow Lowerer', 'Right Eye Wink', 'Left Eye Wink',
           'Lip Puller', 'Chin Raiser', 'Mouth Stretch', 'Chewing', 'Speaking', 'Walking', 'Running']
atoi = {a:i for i,a in enumerate(actions)}
itoa = {i:a for i,a in enumerate(action_name_for_file)}

print(atoi)

In [ ]:
def bandpass_filter(data, lowcut, highcut, fs, order=4):
    '''
    Defines a bandpass filter with nyq being the nyquist, lowcut the lowpass and highcut the highpass frequency. The order markes the order of the bandpass filter.
    '''
    nyq = 0.5 * fs 
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

In [ ]:
def data_loader(patient, action, right_ear, left_ear, sensor_configuration):
    '''
    Automate input reading: select patient, action, imu/ppg, left/right ear, sensor congiration (from 1-4)
    Read in csv file, add sensor configration type and change to ms timestamps
    '''
    df_data_ppg = pd.read_csv(
        '../data/Dataset/P'+ str(patient) + '/EARBUDS/' + str(patient) + '-' + action_name_for_file[action] + '-' + 'ppg' + '-' + ['right' if right_ear==1 else ('left' if left_ear else None)][0] + '.csv',
        sep=',',           # specify delimiter (default is ',')
        header=0,          # row number to use as column names (0 means the first row)
        na_values=['NA', ''],  # specify which values should be considered NaN
    )
    df_data_imu = pd.read_csv(
        '../data/Dataset/P'+ str(patient) + '/EARBUDS/' + str(patient) + '-' + action_name_for_file[action] + '-' + 'imu' + '-' + ['right' if right_ear==1 else ('left' if left_ear else None)][0] + '.csv',
        sep=',',           # specify delimiter (default is ',')
        header=0,          # row number to use as column names (0 means the first row)
        na_values=['NA', ''],  # specify which values should be considered NaN
    )
    print('../data/Dataset/P'+ str(patient) + '/EARBUDS/' + str(patient) + '-' + action_name_for_file[action] + '-' + 'ppg' + '-' + ['right' if right_ear==1 else ('left' if left_ear else None)][0] + '.csv',)

    # Handle IMU signals
    # Convert raw accelerometer data to milli-g units by multiplying with 0.061, the raw gyroscope data to milli-dps (degrees per second) by multiplying with 17.5 and change timesteps
    df_data_imu *= [1, 0.061, 0.061, 0.061, 17.5, 17.5, 17.5]
    df_data_imu['timestamp'] = pd.to_datetime(df_data_imu['timestamp'], unit='ms')
    
    # Convert all imu signals to numpy arrays, change dtype to int and discard the first and last second     
    array_ax = np.array(df_data_imu['ax']).astype(int)[100:-100]
    array_ay = np.array(df_data_imu['ay']).astype(int)[100:-100]
    array_az = np.array(df_data_imu['az']).astype(int)[100:-100]
    array_gx = np.array(df_data_imu['gx']).astype(int)[100:-100]
    array_gy = np.array(df_data_imu['gy']).astype(int)[100:-100]
    array_gz = np.array(df_data_imu['gz']).astype(int)[100:-100]
    
    # Handle PPG signals
    # Add a 'configuration' column initialized with NaN values
    df_data_ppg['sensor_config'] = pd.NA

    # Start with adding sensor configuration index
    config_type = 1
    config_rows = []

    # Loop through rows, change timesteps, store configuration changes and assign configuration index
    for i, row in df_data_ppg.iterrows():
        if row['timestamp'].startswith('#'):
            df_data_ppg.at[i, 'sensor_config'] = 'Config Change'
            config_rows.append(i)
            config_type += 1
        else:
            df_data_ppg.at[i, 'sensor_config'] = config_type
            df_data_ppg.at[i, 'timestamp'] = pd.to_datetime(df_data_ppg.at[i, 'timestamp'], unit='ms')

    # Select the sensor configuration, 
    df_data_sensor_config = df_data_ppg[df_data_ppg['sensor_config'] == sensor_configuration]

    # Convert all ppg signals to numpy arrays, change dtype to int and discard the first and last second
    array_green_sensor_config = np.array(df_data_sensor_config['green']).astype(int)[100:-100]
    array_ir_sensor_config = np.array(df_data_sensor_config['ir']).astype(int)[100:-100]
    array_red_sensor_config = np.array(df_data_sensor_config['red']).astype(int)[100:-100]

    # Print the resulting DataFrame
    print("Time of sensor configuration change:", '\n', df_data_ppg.iloc[config_rows], '\n')
    #print("Added sensor_config and converted timesteps:", '\n',df_data_ppg)
    print("Selected sensor configuration:", '\n', df_data_sensor_config)
    print("The green ppg signal -", "sample size:", '\n', array_green_sensor_config, '-', len(array_green_sensor_config))
    print("The ir ppg signal -", "sample size:", '\n', array_ir_sensor_config, '-', len(array_ir_sensor_config))
    print("The red ppg signal -", "sample size:", '\n', array_red_sensor_config, '-', len(array_red_sensor_config))

    return array_green_sensor_config, array_ir_sensor_config, array_red_sensor_config, array_ax, array_ay, array_az, array_gx, array_gy, array_gz

In [ ]:
green_ppg, ir_ppg, red_ppg, ax, ay, az, gx, gy, gz = data_loader(patient=10, action=16, right_ear=False, left_ear=True, sensor_configuration=4)

## PPG Signals - Analysis

In [ ]:
# Create a subplot figure showing the 3 different ppg signals
downsampling_rate = 1
fig, axes = plt.subplots(1, 3, figsize=(20, 6))  # 1 row, 3 columns

axes[0].plot(green_ppg[::downsampling_rate], label='Green PPG', color='g')
axes[0].set_title("Green PPG Signals")
axes[0].set_xlabel("samples")
axes[0].set_ylabel("arb. unit")
axes[0].legend()

axes[1].plot(ir_ppg[::downsampling_rate], label='IR PPG', color='darkviolet')
axes[1].set_title("IR PPG Signals")
axes[1].set_xlabel("samples")
axes[1].set_ylabel("arb. unit")
axes[1].legend()

axes[2].plot(red_ppg[::downsampling_rate], label='Red PPG', color='r')
axes[2].set_title("Red PPG Signals")
axes[2].set_xlabel("samples")
axes[2].set_ylabel("arb. unit")
axes[2].legend()

plt.show()

### Filtering the PPG signals with a bandpass filter

In [ ]:
# Sample data and sampling frequency
fs = 100  

# Define bandpass range for PPG 
lowcut = 0.5
highcut = 10

# Apply the bandpass filter to the PPG signal
filtered_green_ppg = bandpass_filter(green_ppg, lowcut, highcut, fs, order=4)
filtered_ir_ppg = bandpass_filter(ir_ppg, lowcut, highcut, fs, order=4)
filtered_red_ppg = bandpass_filter(red_ppg, lowcut, highcut, fs, order=4)

# Plot the original and filtered PPG signals
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(green_ppg, color='g', label="Original PPG Signal")
plt.title("Original PPG Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")

plt.subplot(2, 1, 2)
plt.plot(filtered_green_ppg, color='blue', label="Filtered PPG Signal (Bandpass 2-10 Hz)")
plt.title("Filtered PPG Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Create a subplot figure showing the 3 different ppg signals
downsampling_rate = 1
fig, axes = plt.subplots(2, 3, figsize=(20, 6))  # 1 row, 3 columns

axes[0][0].plot(green_ppg[::downsampling_rate], label='Green PPG', color='g')
axes[0][0].set_title("Green PPG Signals")
axes[0][0].set_xlabel("samples")
axes[0][0].set_ylabel("arb. unit")
axes[0][0].legend()

axes[1][0].plot(filtered_green_ppg[::downsampling_rate], label='Filtered Green PPG', color='g')
axes[1][0].set_title("Filtered Green PPG Signals")
axes[1][0].set_xlabel("samples")
axes[1][0].set_ylabel("arb. unit")
axes[1][0].legend()

axes[0][1].plot(ir_ppg[::downsampling_rate], label='IR PPG', color='darkviolet')
axes[0][1].set_title("IR PPG Signals")
axes[0][1].set_xlabel("samples")
axes[0][1].set_ylabel("arb. unit")
axes[0][1].legend()

axes[1][1].plot(filtered_ir_ppg[::downsampling_rate], label='Filtered IR PPG', color='darkviolet')
axes[1][1].set_title("FilteredIR PPG Signals")
axes[1][1].set_xlabel("samples")
axes[1][1].set_ylabel("arb. unit")
axes[1][1].legend()


axes[0][2].plot(red_ppg[::downsampling_rate], label='Red PPG', color='r')
axes[0][2].set_title("Red PPG Signals")
axes[0][2].set_xlabel("samples")
axes[0][2].set_ylabel("arb. unit")
axes[0][2].legend()

axes[1][2].plot(filtered_red_ppg[::downsampling_rate], label='Filtered Red PPG', color='r')
axes[1][2].set_title("FilteredRed PPG Signals")
axes[1][2].set_xlabel("samples")
axes[1][2].set_ylabel("arb. unit")
axes[1][2].legend()

#plt.savefig('../results/data_analysis/ppg_p0_a0_le_sc4.png')
plt.show()

## IMU Signals - Analysis

In [ ]:
# Create a subplot figure showing the 3 different axis accelerometer signals
fig, axes = plt.subplots(1, 3, figsize=(20, 6))  # 1 row, 3 columns

axes[0].plot(ax, label='ax', color='g')
axes[0].set_title("Accelerometer")
axes[0].set_xlabel("samples")
axes[0].set_ylabel("milli-g")
axes[0].legend()

axes[1].plot(ay, label='ay', color='darkviolet')
axes[1].set_title("Accelerometer")
axes[1].set_xlabel("samples")
axes[1].set_ylabel("milli-g")
axes[1].legend()

axes[2].plot(az, label='az', color='r')
axes[2].set_title("Accelerometer")
axes[2].set_xlabel("samples")
axes[2].set_ylabel("milli-g")
axes[2].legend()

plt.show()

In [ ]:
# Create a subplot figure showing the 3 different axis gyroscope signals
fig, axes = plt.subplots(1, 3, figsize=(20, 6))  # 1 row, 3 columns

axes[0].plot(gx, label='ax', color='g')
axes[0].set_title("Gyroscope")
axes[0].set_xlabel("samples")
axes[0].set_ylabel("milli degress per second")
axes[0].legend()

axes[1].plot(gy, label='ay', color='darkviolet')
axes[1].set_title("Gyroscope")
axes[1].set_xlabel("samples")
axes[1].set_ylabel("milli degress per second")
axes[1].legend()

axes[2].plot(gz, label='az', color='r')
axes[2].set_title("Gyroscope")
axes[2].set_xlabel("samples")
axes[2].set_ylabel("milli degress per second")
axes[2].legend()

plt.show()

### Filtering the IMU signals with a bandpass filter

In [ ]:
# Sample data and sampling frequency
fs = 100  

# Define bandpass range for PPG 
lowcut = 0.4
highcut = 10

# Apply the bandpass filter to the IMU signals
ax_filtered= bandpass_filter(ax, lowcut, highcut, fs, order=4)
ay_filtered= bandpass_filter(ay, lowcut, highcut, fs, order=4)
az_filtered = bandpass_filter(az, lowcut, highcut, fs, order=4)
gx_filtered= bandpass_filter(gx, lowcut, highcut, fs, order=4)
gy_filtered= bandpass_filter(gy, lowcut, highcut, fs, order=4)
gz_filtered = bandpass_filter(gz, lowcut, highcut, fs, order=4)

In [ ]:
# Create a subplot figure showing the 3 different axis accelerometer signals
fig, axes = plt.subplots(1, 3, figsize=(20, 6))  # 1 row, 3 columns

axes[0].plot(ax_filtered, label='ax', color='g')
axes[0].set_title("Accelerometer")
axes[0].set_xlabel("samples")
axes[0].set_ylabel("milli-g")
axes[0].legend()

axes[1].plot(ay_filtered, label='ay', color='darkviolet')
axes[1].set_title("Accelerometer")
axes[1].set_xlabel("samples")
axes[1].set_ylabel("milli-g")
axes[1].legend()

axes[2].plot(az_filtered, label='az', color='r')
axes[2].set_title("Accelerometer")
axes[2].set_xlabel("samples")
axes[2].set_ylabel("milli-g")
axes[2].legend()

plt.show()

In [ ]:
# Create a subplot figure showing the 3 different imu signals
downsampling_rate = 1
fig, axes = plt.subplots(2, 3, figsize=(20, 6))  # 1 row, 3 columns

axes[0][0].plot(ax[::downsampling_rate], label='ax', color='g')
axes[0][0].set_title("Accelerometer")
axes[0][0].set_xlabel("samples")
axes[0][0].set_ylabel("milli-g")
axes[0][0].legend()

axes[1][0].plot(ax_filtered[::downsampling_rate], label='Filtered ax', color='g')
axes[1][0].set_title("Accelerometer")
axes[1][0].set_xlabel("samples")
axes[1][0].set_ylabel("milli-g")
axes[1][0].legend()

axes[0][1].plot(ay[::downsampling_rate], label='ay', color='darkviolet')
axes[0][1].set_title("Accelerometer")
axes[0][1].set_xlabel("samples")
axes[0][1].set_ylabel("milli-g")
axes[0][1].legend()

axes[1][1].plot(ay_filtered[::downsampling_rate], label='Filtered ay', color='darkviolet')
axes[1][1].set_title("Accelerometer")
axes[1][1].set_xlabel("samples")
axes[1][1].set_ylabel("milli-g")
axes[1][1].legend()


axes[0][2].plot(az[::downsampling_rate], label='az', color='r')
axes[0][2].set_title("Accelerometer")
axes[0][2].set_xlabel("samples")
axes[0][2].set_ylabel("milli-g")
axes[0][2].legend()

axes[1][2].plot(az_filtered[::downsampling_rate], label='Filtered az', color='r')
axes[1][2].set_title("Accelerometer")
axes[1][2].set_xlabel("samples")
axes[1][2].set_ylabel("milli-g")
axes[1][2].legend()

#plt.savefig('../results/data_analysis/ppg_p0_a0_le_sc4.png')
plt.show()

In [ ]:
# Create a subplot figure showing the 3 different imu signals
downsampling_rate = 1
fig, axes = plt.subplots(2, 3, figsize=(20, 6))  # 1 row, 3 columns

axes[0][0].plot(gx[::downsampling_rate], label='gx', color='g')
axes[0][0].set_title("Gyroscope")
axes[0][0].set_xlabel("samples")
axes[0][0].set_ylabel("milli degress per second")
axes[0][0].legend()

axes[1][0].plot(gx_filtered[::downsampling_rate], label='Filtered gx', color='g')
axes[1][0].set_title("Gyroscope")
axes[1][0].set_xlabel("samples")
axes[1][0].set_ylabel("milli degress per second")
axes[1][0].legend()

axes[0][1].plot(gy[::downsampling_rate], label='gy', color='darkviolet')
axes[0][1].set_title("Gyroscope")
axes[0][1].set_xlabel("samples")
axes[0][1].set_ylabel("milli degress per secondg")
axes[0][1].legend()

axes[1][1].plot(gy_filtered[::downsampling_rate], label='Filtered gy', color='darkviolet')
axes[1][1].set_title("Gyroscope")
axes[1][1].set_xlabel("samples")
axes[1][1].set_ylabel("milli degress per second")
axes[1][1].legend()


axes[0][2].plot(gz[::downsampling_rate], label='gz', color='r')
axes[0][2].set_title("Gyroscope")
axes[0][2].set_xlabel("samples")
axes[0][2].set_ylabel("milli degress per second")
axes[0][2].legend()

axes[1][2].plot(gz_filtered[::downsampling_rate], label='Filtered gz', color='r')
axes[1][2].set_title("Gyroscope")
axes[1][2].set_xlabel("samples")
axes[1][2].set_ylabel("milli degress per second")
axes[1][2].legend()

#plt.savefig('../results/data_analysis/ppg_p0_a0_le_sc4.png')
plt.show()

# Zephyr Signals - Analysis


In [ ]:
def zephyr_data_loader(patient):
    '''
    Automate input reading: select patient
    Read in csv file
    '''
    df_data = pd.read_csv(
        '../data/Dataset/P'+ str(patient) + '/ZEPHYR/' + str(patient) + '_' + 'Summary.csv',
        sep=',',           # specify delimiter (default is ',')
        header=0,          # row number to use as column names (0 means the first row)
        na_values=['NA', ''],  # specify which values should be considered NaN
    )
    print('../data/Dataset/P'+ str(patient) + '/ZEPHYR/' + str(patient) + '_' + 'Summary.csv',)
    # Change timesteps
    df_data['Timestamp'] = pd.to_datetime(df_data['Timestamp'], unit='ms')
    print(df_data)
    
    

    # Add an 'action' column initialized with NaN values
    #df_data['action'] = pd.NA

    #     # Start with adding sensor configuration index
    #     config_type = 1
    #     config_rows = []

    #     # Loop through rows, change timesteps, store configuration changes and assign configuration index
    #     for i, row in df_data.iterrows():
    #         if row['timestamp'].startswith('#'):
    #             df_data.at[i, 'sensor_config'] = 'Config Change'
    #             config_rows.append(i)
    #             config_type += 1
    #         else:
    #             df_data.at[i, 'sensor_config'] = config_type
    #             df_data.at[i, 'timestamp'] = pd.to_datetime(df_data.at[i, 'timestamp'], unit='ms')

    #     # Select the sensor configuration, 
    #     df_data_sensor_config = df_data[df_data['sensor_config'] == sensor_configuration]
    #     # Convert all ppg signals to numpy arrays, change dtype to int and discard the first and last second
    array_ecg = np.array(df_data['ECGAmplitude'].astype(float))[100:-100]
    array_hr = np.array(df_data['HR'].astype(int))[100:-100]
    
    #     array_ir_sensor_config = np.array(df_data_sensor_config['ir']).astype(int)[100:-100]
    #     array_red_sensor_config = np.array(df_data_sensor_config['red']).astype(int)[100:-100]

    #     # Print the resulting DataFrame
    #     print("Time of sensor configuration change:", '\n', df_data.iloc[config_rows], '\n')
    #     #print("Added sensor_config and converted timesteps:", '\n',df_data)
    #     print("Selected sensor configuration:", '\n', df_data_sensor_config)
    #     print("The green ppg signal -", "sample size:", '\n', array_green_sensor_config, '-', len(array_green_sensor_config))
    #     print("The ir ppg signal -", "sample size:", '\n', array_ir_sensor_config, '-', len(array_ir_sensor_config))
    #     print("The red ppg signal -", "sample size:", '\n', array_red_sensor_config, '-', len(array_red_sensor_config))

    return array_hr, array_ecg

In [ ]:
hr, ecg = zephyr_data_loader(patient=0)
print(hr, ecg)

In [ ]:
# Plot the ECG signal
plt.figure(figsize=(12, 6))
plt.plot(ecg[:1000], color='b', label="ECG Signal",)
plt.title("Zephyr ECG Signal")
plt.xlabel("Time (s)")
plt.ylabel("ECG")
plt.legend()
plt.tight_layout()

plt.show()

In [ ]:
# Plot the ECG signal
plt.figure(figsize=(12, 6))
plt.plot(hr[:1000], color='b', label="HR Signal", marker='o')
plt.title("Zephyr HR Signal")
plt.xlabel("Time (s)")
plt.ylabel("HR")
plt.legend()
plt.tight_layout()